## Hypothesis: Flod risk preception has a positive effect on preparedness

In [5]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [6]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df = pd.read_csv(path)
df.head()

,id,state,zipcode,geographic_division,census_region,county,awareness,perception,experience,floodzone,...,sex,education,race,homeownership,income,rentmortgage,rurality,hazard_weight,geometry,zip_count
0,1,Washington,98033,Pacific,West,King,0.0,1.0,0.0,0.0,...,0.0,4,White,1,87500,1250.0,1,0.255097,POLYGON ((-13608075.601186747 6055357.24645823...,1
1,2,Florida,33950,South Atlantic,South,Charlotte,1.0,1.0,1.0,1.0,...,1.0,5,White,1,175000,3000.0,1,2.836145,POLYGON ((-9139287.781401768 3112400.961508024...,1
2,3,New Jersey,7031,Middle Atlantic,Northeast,Bergen,0.0,1.0,1.0,0.0,...,1.0,1,White,0,47500,1250.0,1,1.165971,POLYGON ((-8254065.171859486 4980899.960122439...,1
3,5,Pennsylvania,19148,Middle Atlantic,Northeast,Philadelphia,0.0,1.0,1.0,0.0,...,0.0,4,White,1,125000,1250.0,1,0.624890,POLYGON ((-8368433.592186097 4851218.465580212...,1
4,8,New York,11691,Middle Atlantic,Northeast,Queens,1.0,1.0,1.0,1.0,...,0.0,3,Black or African American,1,62500,1750.0,1,0.853837,POLYGON ((-8214183.071851451 4953007.332301349...,2


In [7]:
df.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication


In [8]:
# define the regression model 
def model (function, data):
    model = smf.logit(function, data=data).fit()
    print(model.summary()) # print model summary
    print('BIC:', model.bic) # print BIC additionally to defaults model fit parameters
    # print odds ratios for output = 0 and output = 1
    print('likelihood of adaptation when there is low risk perception = ', 1 / (1 + np.exp(-model.params['Intercept'])))
    print('likelihood of adaptation when there is high risk perception = ', 1 / (1 + np.exp(-(model.params['Intercept']+model.params['perception']))))

    # compute odds ratio
    params = model.params
    conf = model.conf_int()
    odds_ratios = pd.DataFrame({
        "OR": np.exp(params),
        "Lower CI": np.exp(conf[0]),
        "Upper CI": np.exp(conf[1])
        })
    odds_ratios.index.name = 'Variable'
    odds_ratios.reset_index(inplace=True)
    # compute marginal effects
    marginal_effects = model.get_margeff().summary()

    return odds_ratios, marginal_effects 
    

In [9]:
odds, marg_eff = model('made_safer ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.464444
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:             made_safer   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.05101
Time:                        17:16:52   Log-Likelihood:                -178.81
converged:                       True   LL-Null:                       -188.42
Covariance Type:            nonrobust   LLR p-value:                 1.163e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2513      0.263     -8.566      0.000      -2.766      -1.736
perception     1.2428      0.

In [10]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.105263,0.062886,0.176196
1,perception,3.465409,1.908615,6.292029


In [11]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:             made_safer
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.1839      0.044      4.226      0.000       0.099       0.269
==============================================================================
"""

In [12]:
odds, marg_eff = model('insured ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.483293
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                insured   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                  0.1211
Time:                        17:16:52   Log-Likelihood:                -186.07
converged:                       True   LL-Null:                       -211.70
Covariance Type:            nonrobust   LLR p-value:                 8.043e-13
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.5649      0.300     -8.562      0.000      -3.152      -1.978
perception     2.0270      0.

In [13]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.076923,0.042762,0.138374
1,perception,7.591241,3.968153,14.522359


In [14]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                insured
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.3246      0.046      7.033      0.000       0.234       0.415
==============================================================================
"""

In [15]:
odds, marg_eff = model('learned_routes ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.522120
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:         learned_routes   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.07502
Time:                        17:16:53   Log-Likelihood:                -201.02
converged:                       True   LL-Null:                       -217.32
Covariance Type:            nonrobust   LLR p-value:                 1.129e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0595      0.244     -8.454      0.000      -2.537      -1.582
perception     1.4817      0.

In [16]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.127517,0.079106,0.205553
1,perception,4.400606,2.533522,7.643641


In [17]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:         learned_routes
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.2571      0.044      5.838      0.000       0.171       0.343
==============================================================================
"""

In [18]:
odds, marg_eff = model('supplies ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.496299
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               supplies   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.09244
Time:                        17:16:53   Log-Likelihood:                -191.08
converged:                       True   LL-Null:                       -210.54
Covariance Type:            nonrobust   LLR p-value:                 4.402e-10
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.3224      0.271     -8.584      0.000      -2.853      -1.792
perception     1.7044      0.

In [19]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.098039,0.057690,0.166610
1,perception,5.497872,3.019827,10.009382


In [20]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               supplies
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.2791      0.045      6.215      0.000       0.191       0.367
==============================================================================
"""

In [21]:
odds, marg_eff = model('involved ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.285588
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:               involved   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                  0.1291
Time:                        17:16:53   Log-Likelihood:                -109.95
converged:                       True   LL-Null:                       -126.25
Covariance Type:            nonrobust   LLR p-value:                 1.131e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.4188      0.711     -6.212      0.000      -5.813      -3.025
perception     2.8368      0.

In [22]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.012048,0.002988,0.048577
1,perception,17.061111,4.048666,71.895657


In [23]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:               involved
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.2407      0.066      3.668      0.000       0.112       0.369
==============================================================================
"""

In [24]:
odds, marg_eff = model('made_plan ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.512169
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:              made_plan   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.06342
Time:                        17:16:53   Log-Likelihood:                -197.18
converged:                       True   LL-Null:                       -210.54
Covariance Type:            nonrobust   LLR p-value:                 2.369e-07
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0595      0.244     -8.454      0.000      -2.537      -1.582
perception     1.3594      0.

In [25]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.127517,0.079106,0.205553
1,perception,3.894011,2.235877,6.781824


In [26]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:              made_plan
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.2294      0.044      5.189      0.000       0.143       0.316
==============================================================================
"""

In [27]:
odds, marg_eff = model('practiced_drills ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.265542
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:       practiced_drills   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.09219
Time:                        17:16:53   Log-Likelihood:                -102.23
converged:                       True   LL-Null:                       -112.62
Covariance Type:            nonrobust   LLR p-value:                 5.194e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.0073      0.583     -6.879      0.000      -5.149      -2.866
perception     2.1774      0.

In [28]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.018182,0.005804,0.056954
1,perception,8.823529,2.643975,29.446073


In [29]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:       practiced_drills
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.1629      0.049      3.303      0.001       0.066       0.260
==============================================================================
"""

In [30]:
odds, marg_eff = model('alerts ~ perception', data = df)

Optimization terminated successfully.
         Current function value: 0.460683
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 alerts   No. Observations:                  385
Model:                          Logit   Df Residuals:                      383
Method:                           MLE   Df Model:                            1
Date:                Mon, 07 Apr 2025   Pseudo R-squ.:                 0.07938
Time:                        17:16:53   Log-Likelihood:                -177.36
converged:                       True   LL-Null:                       -192.65
Covariance Type:            nonrobust   LLR p-value:                 3.197e-08
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.4785      0.289     -8.584      0.000      -3.044      -1.913
perception     1.6069      0.

In [31]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.083871,0.047624,0.147704
1,perception,4.987431,2.638478,9.427583


In [32]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
        Logit Marginal Effects       
=====================================
Dep. Variable:                 alerts
Method:                          dydx
At:                           overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.2384      0.045      5.246      0.000       0.149       0.327
==============================================================================
"""

In [33]:
odds, marg_eff = model('family_communication ~ perception', data=df)

Optimization terminated successfully.
         Current function value: 0.324597
         Iterations 7
                            Logit Regression Results                            
Dep. Variable:     family_communication   No. Observations:                  385
Model:                            Logit   Df Residuals:                      383
Method:                             MLE   Df Model:                            1
Date:                  Mon, 07 Apr 2025   Pseudo R-squ.:                 0.05808
Time:                          17:16:53   Log-Likelihood:                -124.97
converged:                         True   LL-Null:                       -132.68
Covariance Type:              nonrobust   LLR p-value:                 8.651e-05
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.1355      0.386     -8.121      0.000      -3.892      -2.379
perception   

In [34]:
odds

,Variable,OR,Lower CI,Upper CI
0,Intercept,0.043478,0.020400,0.092665
1,perception,4.423077,1.911876,10.232677


In [35]:
marg_eff

<class 'statsmodels.iolib.summary.Summary'>
"""
         Logit Marginal Effects        
=======================================
Dep. Variable:     family_communication
Method:                            dydx
At:                             overall
==============================================================================
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
perception     0.1393      0.041      3.359      0.001       0.058       0.221
==============================================================================
"""